<a href="https://colab.research.google.com/github/subhamchatterjee21/Toxic-Text-Detector/blob/main/Toxic_Text_Derector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv("FinalBalancedDataset.csv")


In [ ]:
data.info()

In [ ]:
data.head(5)

In [ ]:
data = data.drop("Unnamed: 0", axis=1)

In [ ]:
data['Toxicity'].value_counts()

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
import re

In [ ]:
def prepare_text(text):
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    text = text.split()
    text = ' '.join(text)
    text = word_tokenize(text)
    text = pos_tag(text)
    lemma = []
    for i in text: lemma.append(wordnet_lemmatizer.lemmatize(i[0], pos = get_wordnet_pos(i[1])))
    lemma = ' '.join(lemma)
    return lemma

In [ ]:
data['clean_tweets'] = data['tweet'].apply(lambda x: prepare_text(x))

In [ ]:
data.head(5)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score


In [ ]:
corpus = data['clean_tweets'].values.astype('U')

In [ ]:
stopwords = set(nltk_stopwords.words('english'))

In [ ]:
stopwords_list=list(stopwords)

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords_list)
tf_idf = count_tf_idf.fit_transform(corpus)

In [ ]:
import pickle

In [ ]:
pickle.dump(count_tf_idf, open("tf_idf.pkt", "wb"))

In [ ]:
tf_idf_train, tf_idf_test, target_train, target_test = train_test_split(
    tf_idf, data['Toxicity'], test_size = 0.2, random_state= 42, shuffle=True
)

# clssifier

In [ ]:
model = MultinomialNB()

In [ ]:
model.fit(tf_idf_train, target_train)

In [ ]:
y_pred = model.predict(tf_idf_test)

In [ ]:
pickle.dump(model, open("toxicity_classification_model.pkt", "wb"))

In [ ]:
test_text = "I will kill you"
test_tfidf = count_tf_idf.transform([test_text])
predicted_toxicity = model.predict(test_tfidf)
print(predicted_toxicity)

[1]


In [ ]:
accuracy = accuracy_score(target_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9154991629218433
